In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/salary-prediction-for-job-postings/usjobs_train.csv
/kaggle/input/salary-prediction-for-job-postings/usjobs_sample_submission.csv
/kaggle/input/salary-prediction-for-job-postings/usjobs_test.csv


## Salary Prediction for Job Postings

This competition is an in-class competition for the course, "Artificial Intelligence and Machine Learning (Fall 2023)", in Renmin University of China.

## Content

- [packages](#pack)
- [load data](#load)
- [engineering](#engineer)
- [model](#model)
- [submit](#sub)

## <a id = 'pack'> Packages </a>

In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt


## <a id = 'load'> Load Data </a>

In [3]:
train = pd.read_csv('/kaggle/input/salary-prediction-for-job-postings/usjobs_train.csv')
print(train.shape)
train.sample(3)

(33248, 21)


,ID,Job,Jobs_Group,Profile,Remote,Company,Location,City,State,Frecuency_Salary,...,Skills,Sector,Sector_Group,Revenue,Employee,Company_Score,Reviews,Director,Director_Score,URL
5399,job_f1b17a1e090d4400,Senior Budget Analyst,Financial Analyst,Senior,Hybrid,WellLife Network,"Hempstead, NY",Hempstead,NY,year,...,[],NGOs and Nonprofit Organizations,Nonprofit Organizations,M,XL,3.0,100.0,Sherry Tucker,0.59,http://www.welllifenetwork.org/
25367,job_5fcbecf6041e7b1f,Information Technology Business Analyst,Business Analyst,NaN,NaN,Craftner INC,"Phoenix, NY 13135",Phoenix,NY,hour,...,"['PowerPoint', 'Jira', 'Excel', 'Agile']",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32625,job_1572bf0c4daf4008,Economist / Statistician (Economics),Statistician/Mathemathics,NaN,NaN,US Bureau of Labor Statistics,"Philadelphia, PA+1 ubicación",Philadelphia,PA,year,...,"['English', 'Office', 'Bachelor']",Government and Public Administration,Government,NaN,XL,4.2,11.0,NaN,NaN,NaN


In [4]:
test = pd.read_csv('/kaggle/input/salary-prediction-for-job-postings/usjobs_test.csv')
print(test.shape)
test.sample(3)

(22166, 20)


,ID,Job,Jobs_Group,Profile,Remote,Company,Location,City,State,Frecuency_Salary,Skills,Sector,Sector_Group,Revenue,Employee,Company_Score,Reviews,Director,Director_Score,URL
6677,job_a77b4cd495e1de10,Associate Big Data Engineer,Data Engineer,NaN,NaN,Highmark Health,Pennsylvania+1 ubicación,NaN,PA,year,"['English', 'Python', 'SQL', 'Google Cloud', '...",Healthcare,Health,XXXL,XXXL,3.2,937.0,David L. Holmberg,0.64,https://www.highmarkhealth.org/
6225,job_0cf79db4cf486181,Financial Analyst II,Financial Analyst,NaN,NaN,Plaxonic Technologies Inc.,"Irving, TX",Irving,TX,hour,"['Office', 'Bachelor']",Transportation and Logistics,Transportation and Logistics,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6386,job_8110e9ac93139e60,OAG - Child Support | IT Business Analyst I-II...,Business Analyst,NaN,Remote,Office of the Attorney General of Texas,"Austin, TX 78741 (Pleasant Valley area)",Austin,TX,month,"['Office', 'Bachelor']",NaN,NaN,NaN,NaN,3.5,13.0,NaN,NaN,NaN


In [5]:
submission = pd.read_csv('/kaggle/input/salary-prediction-for-job-postings/usjobs_sample_submission.csv')
print(submission.shape)
submission.sample(3)

(22166, 2)


,ID,Mean_Salary
6018,job_0d3eff11444ab89d,105000
949,job_37660a1c31c803ed,105000
21122,job_113b14d2493325c0,105000


## <a id = 'engineer'> Engineering </a>

In [6]:
train.isna().sum()

ID                      0
Job                     0
Jobs_Group              0
Profile             21107
Remote              19319
Company                 9
Location               13
City                 3824
State                3112
Frecuency_Salary        0
Mean_Salary             0
Skills                  0
Sector               7214
Sector_Group         7214
Revenue             18318
Employee            12799
Company_Score        8762
Reviews              8762
Director            20785
Director_Score      21924
URL                 16033
dtype: int64

In [7]:
def rearrange_dataset(train_data, test_data):
    #1.train 
    skills_list = []
    for i in range(len(train_data)):
        zip_list = train_data['Skills'][i].replace('[','').replace(']','').replace("'",'').replace(" ",'').split(',')
        dict_temp = {}
        str_ = ''
        for j in range(len(zip_list)):
            dict_temp[zip_list[j]] = 1
            str_ += zip_list[j] + " "
        dict_temp['skills_clean'] = str_    
        skills_list.append(dict_temp)
        
    train_data = pd.concat([train_data,pd.DataFrame(skills_list).fillna(0)],axis =1)  
    
    
    #2.test
    skills_list = []
    for i in range(len(test_data)):
        zip_list = test_data['Skills'][i].replace('[','').replace(']','').replace("'",'').replace(" ",'').split(',')
        dict_temp = {}
        str_ = ''
        for j in range(len(zip_list)):
            dict_temp[zip_list[j]] = 1
            str_ += zip_list[j] + " "
        dict_temp['skills_clean'] = str_        
        skills_list.append(dict_temp)
        
    test_data = pd.concat([test_data,pd.DataFrame(skills_list).fillna(0)],axis =1)      
    
    return train_data, test_data

In [8]:
%%time
train_data1, test_data1 = rearrange_dataset(train_data= train, test_data= test)

CPU times: user 2.41 s, sys: 105 ms, total: 2.51 s
Wall time: 2.49 s


In [9]:
train_data1[['Job','Company','skills_clean','ArtificialIntelligence', 'MachineLearning', 'Python', 'TensorFlow',
       'Excel', 'Spark', 'PhD', 'AWS', 'C++', 'DeepLearning', 'Java', 'Master',
       'Office', 'Docker', 'Word', 'Azure', 'Hadoop', 'Airflow', 'MBA', 'CPA',
       'Director_Score', 'Snowflake', 'Databricks', 'SQL', '', 'Company_Score',
       'GoogleCloud', 'SciKit', 'Reviews', 'Pandas', 'ChatGPT', 'Agile', 'R',
       'NeuralNetwork', 'Jupyter', 'PowerPoint', 'Spanish', 'NumPy', 'React',
       'GitHub', 'Looker', 'English', 'Access', 'Matplotlib', 'Seaborn', 'C#',
       'Rust', 'VBA', 'Bachelor']].isna().sum()

Job                           0
Company                       9
skills_clean                  0
ArtificialIntelligence        0
MachineLearning               0
Python                        0
TensorFlow                    0
Excel                         0
Spark                         0
PhD                           0
AWS                           0
C++                           0
DeepLearning                  0
Java                          0
Master                        0
Office                        0
Docker                        0
Word                          0
Azure                         0
Hadoop                        0
Airflow                       0
MBA                           0
CPA                           0
Director_Score            21924
Snowflake                     0
Databricks                    0
SQL                           0
                              0
Company_Score              8762
GoogleCloud                   0
SciKit                        0
Reviews 

In [10]:
train_data1['Director_Score'].fillna(np.mean(train_data1['Director_Score']), inplace= True)
train_data1['Company_Score'].fillna(np.mean(train_data1['Company_Score']), inplace= True)
train_data1['Reviews'].fillna(np.mean(train_data1['Reviews']), inplace= True)

## <a id = 'model'> Model </a>

In [11]:
X_train, X_test, y_train, y_test = train_test_split(train_data1[['ArtificialIntelligence', 'MachineLearning', 'Python', 'TensorFlow',
       'Excel', 'Spark', 'PhD', 'AWS', 'C++', 'DeepLearning', 'Java', 'Master',
       'Office', 'Docker', 'Word', 'Azure', 'Hadoop', 'Airflow', 'MBA', 'CPA',
       'Director_Score', 'Snowflake', 'Databricks', 'SQL', '', 'Company_Score',
       'GoogleCloud', 'SciKit', 'Reviews']],
                                                   train_data1['Mean_Salary'],
                                                   test_size = 0.2)

In [12]:
clf = RandomForestRegressor(n_estimators= 500,
                            max_depth = 30,
                            min_samples_leaf = 25,
                           n_jobs= 5)

clf.fit(X_train,
       y_train)

RandomForestRegressor(max_depth=30, min_samples_leaf=25, n_estimators=500,
                      n_jobs=5)

In [13]:
mean_absolute_error(y_test,clf.predict(X_test))

27037.521890453256

## <a id = 'sub'> Submission </a>

In [14]:
test_data1['Director_Score'].fillna(np.mean(train_data1['Director_Score']), inplace= True)
test_data1['Company_Score'].fillna(np.mean(train_data1['Company_Score']), inplace= True)
test_data1['Reviews'].fillna(np.mean(train_data1['Reviews']), inplace= True)

In [15]:
test_df = test_data1[['ID','ArtificialIntelligence', 'MachineLearning', 'Python', 'TensorFlow',
       'Excel', 'Spark', 'PhD', 'AWS', 'C++', 'DeepLearning', 'Java', 'Master',
       'Office', 'Docker', 'Word', 'Azure', 'Hadoop', 'Airflow', 'MBA', 'CPA',
       'Director_Score', 'Snowflake', 'Databricks', 'SQL', '', 'Company_Score',
       'GoogleCloud', 'SciKit', 'Reviews']]

In [16]:
pred = clf.predict(test_df.drop('ID', axis = 1))

In [17]:
test_df['Mean_Salary'] = pred

/tmp/ipykernel_19/1862751176.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Mean_Salary'] = pred


In [18]:
test_df[['ID','Mean_Salary']].to_csv('submission.csv', index = False)